# 12 abr 2018

Com os dados de candidatos migrados com sucesso, faremos uma alteração no método de migração para aproveitarmos a linha em que estaremos. Assim, cada loop deverá cadastrar **candidato** e **candidatura** de uma só vez

To do list:
- [ ] Transformar arquivos em classes
- [ ] Instaciar os valore possíveis baseado nas tabelas auxiliares

In [1]:
import utils.tse_migrar_dados_por_estado as mdpe

In [2]:
mdpe.migrar_dados(2010)

Iniciando ../temp\consulta_cand_2010_BR.txt
Total de candidaturas em 2010: 22
22 de 22
Tempo total desse estado: 0.18s 

Tempo total em 2010: 0.18s 



In [3]:
mdpe.migrar_dados(2014)

Iniciando ../temp\consulta_cand_2014_BR.txt
Total de candidaturas em 2014: 28
28 de 28
Tempo total desse estado: 0.19s 

Tempo total em 2014: 0.19s 



In [4]:
anos = [2004, 2006, 2008, 2010, 2012, 2014, 2016]

for ano in anos:
    mdpe.migrar_dados(ano)

Iniciando ../temp\consulta_cand_2004_AC.txt
Total de candidaturas em 2004: 1511
1511 de 1511
Tempo total desse estado: 8.67s 

Iniciando ../temp\consulta_cand_2004_AL.txt
Total de candidaturas em 2004: 7041
1533 de 7041Pandas(Index=1533, nome='VALBER DE FREITAS MATIAS', data_nascimento='02031961', cpf='31801536449', titulo_eleitoral='022331511740', cidade_nascimento='RECIFE', sigla_estado_nascimento='PE', nacionalidade='1', grau_instrucao='8', ocupacao='999', ano='2004', turno='1', descricao_eleicao='ELEICOES 2004', sigla_cidade_eleicao='27456', cidade_eleicao='CORURIPE', sigla_estado_eleicao='AL', cargo='13', numero_candidato='40111', nome_urna='DR. VALBER', situacao_candidatura='2', partido='40', legenda_composicao='PL / PSB / PSDB', legenda_nome='PSB/PSDB/PL', despesa_maxima='82000080000', resultado_candidatura='2')


DataError: 1264 (22003): Out of range value for column 'despesa_maxima' at row 1

In [3]:
import pandas as pd
import numpy as np
import mysql.connector as mysql

BD_CONFIG = {
    'user': 'root',
    'password': '',
    'host': '127.0.0.1',
    'database': 'laddres'
}

cnx = mysql.connect(**BD_CONFIG)
cursor = cnx.cursor(buffered=True)

query = ("SELECT id, sigla, nome FROM estado")

cursor.execute(query)

estados = cursor.fetchall()

In [7]:
# df = pd.read_sql(query, connection,  params=(start_date, end_date))

estados_df = pd.read_sql(query, cnx)

In [56]:
estados_df[estados_df['sigla'] == 'SE'].set_index("id", drop = True)
print(sergipe.index[0])

# print(len(sergipe))

# print('meu id é {}'.format(id))

25


In [68]:
def get_estado_id(sigla):
    estado = estados_df[estados_df['sigla'] == sigla]
    if (len(estado) == 1):
        return estado['id'].iloc[0]
    else:
        return None

print(get_estado_id('SE'))

25


In [79]:
import sys
import os
import glob
import time

import pandas as pd
import numpy as np
import mysql.connector as mysql

import utils.tse_parsers as parsers
import utils.tse_data_headers as headers

class MigrarDados:

    PREFIXO_ARQUIVO_CANDIDATURAS = 'consulta_cand_'
    DIRETORIO_TEMPORARIO = os.path.join('../temp')
    BD_CONFIG = {
        'user': 'root',
        'password': '',
        'host': '127.0.0.1',
        'database': 'laddres'
    }

    def __init__(self):
        self.cnx = mysql.connect(**BD_CONFIG)

        self.estados = self._load(tabela='estado')

    def print_estados(self):
        print(self.estados)

    def _load(self, tabela):
        query = ("SELECT * FROM " + tabela)
        return pd.read_sql(query, self.cnx)


In [80]:
migrar = MigrarDados()
migrar.print_estados()

    id sigla                 nome
0    1    AC                 Acre
1    2    AL              Alagoas
2    3    AM             Amazonas
3    4    AP                Amapá
4    5    BA                Bahia
5    6    CE                Ceará
6    7    DF     Distrito Federal
7    8    ES      Espírito Santos
8    9    GO                Goiás
9   10    MA             Maranhão
10  11    MG         Minas Gerais
11  12    MS   Mato Grosso do Sul
12  13    MT          Mato Grosso
13  14    PA                 Pará
14  15    PB              Paraíba
15  16    PE           Pernambuco
16  17    PI                Piauí
17  18    PR               Paraná
18  19    RJ       Rio de Janeiro
19  20    RN  Rio Grande do Norte
20  21    RO             Rondônia
21  22    RR              Roraima
22  23    RS    Rio Grande do Sul
23  24    SC       Santa Catarina
24  25    SE              Sergipe
25  26    SP            São Paulo
26  27    TO            Tocantins


In [107]:
import sys
import os
import glob
import time

import pandas as pd
import numpy as np
import mysql.connector as mysql

import utils.tse_parsers as parsers
import utils.tse_data_headers as headers

class MigracaoDadosTSE:
    PREFIXO_ARQUIVO_CANDIDATURAS = 'consulta_cand_'
    DIRETORIO_TEMPORARIO = os.path.join('../temp')
    BD_CONFIG = {
        'user': 'root',
        'password': '',
        'host': '127.0.0.1',
        'database': 'laddres'
    }

    def __init__(self):
        self.cnx = mysql.connect(**BD_CONFIG)

        self.cidades = self._load(tabela='cidade')
        self.eleicoes = self._load(tabela='eleicao')
        self.estados = self._load(tabela='estado')
        self.partidos = self._load(tabela='partido')

        self.cnx.close()

    def __del__(self):
        print('destrutor')

    def migrar(self, ano):
        self.cnx = mysql.connect(**BD_CONFIG)

        tempo_inicio_ano = time.time()

        arquivos = self.PREFIXO_ARQUIVO_CANDIDATURAS + str(ano) + '*.txt'
        pathes = os.path.join(self.DIRETORIO_TEMPORARIO, arquivos)
        arquivos_do_ano = sorted(glob.glob(pathes))

        for arquivo_estado in arquivos_do_ano:
            tempo_inicio_estado = time.time()
            print('Iniciando {}'.format(arquivo_estado))

            df_estado = pd.read_csv(
                arquivo_estado,
                sep = ';',
                header = None,
                dtype = np.str,
                names = headers.get_header(ano),
                encoding = 'iso-8859-1'
            )

            candidaturas = (
                df_estado[self._get_colunas_candidatura(ano)]
                .rename(columns = self._get_map_colunas_candidatura(ano))
            )

            total_candidaturas = candidaturas.shape[0]
            print('Total de candidaturas em {}: {}'.format(ano, total_candidaturas))

            for candidatura in candidaturas.itertuples():
                cursor = self.cnx.cursor(buffered=True)

                estado_id = self._get_estado_id(sigla_estado=candidatura.sigla_estado_nascimento)
                cidade_id = None if (not estado_id) else self._get_cidade_id(
                    cidade=candidatura.cidade_nascimento,
                    estado_id=estado_id)
                email = parsers.parse_email(candidatura.email) if ('email' in candidaturas.columns) else None

                # Cadastrar candidato
                query = ("INSERT INTO candidato "
                               "(nome, data_nascimento, cpf, titulo_eleitoral, email, cidade_id, ocupacao_id, nacionalidade_id, grau_instrucao_id) "
                               "VALUES (%(nome)s, %(data_nascimento)s, %(cpf)s, %(titulo_eleitoral)s, %(email)s, %(cidade_id)s, %(ocupacao)s, %(nacionalidade)s, %(grau_instrucao)s)")
                atributos = {
                    'nome': parsers.parse_nome(candidatura.nome),
                    'data_nascimento': parsers.parse_data(candidatura.data_nascimento),
                    'cpf': parsers.parse_cpf(candidatura.cpf),
                    'titulo_eleitoral': parsers.parse_titulo_eleitoral(candidatura.titulo_eleitoral),
                    'email': email,
                    'cidade_id': cidade_id,
                    'ocupacao': parsers.parse_ocupacao(candidatura.ocupacao),
                    'nacionalidade': parsers.parse_nacionalidade(candidatura.nacionalidade),
                    'grau_instrucao': parsers.parse_grau_instrucao(candidatura.grau_instrucao)
                }

                try:
                    cursor.execute(query, atributos)
                except Exception as e:
                    cursor.close()
                    self._handle_exception(exception=e, row=candidatura)

                # Cadastrar candidatura
                candidato_id = cursor.lastrowid
                eleicao_id = self._get_eleicao_id(
                    ano=candidatura.ano,
                    descricao=candidatura.descricao_eleicao)
                estado_id = self._get_estado_id(
                    sigla_estado=candidatura.sigla_estado_eleicao)
                cidade_id = None if (not estado_id) else self._get_cidade_id(
                    cidade=candidatura.cidade_eleicao,
                    estado_id=estado_id)
                partido_id = self._get_partido_id(
                    numero=candidatura.numero_candidato)

                query = ("INSERT INTO candidatura "
                         "(eleicao_id, turno, candidato_id, cidade_id, estado_id, numero_candidato, nome_urna, partido_id, legenda_nome, legenda_composicao, cargo_id, despesa_maxima, situacao_candidatura_id, resultado_candidatura_id) "
                         "VALUES (%(eleicao)s, %(turno)s, %(candidato)s, %(cidade)s, %(estado)s, %(numero_candidato)s, %(nome_urna)s, %(partido)s, %(legenda_nome)s, %(legenda_composicao)s, %(cargo)s, %(despesa_maxima)s, %(situacao_candidatura)s, %(resultado_candidatura)s)")
                atributos = {
                    'eleicao': eleicao_id,
                    'turno': parsers.parse_turno(candidatura.turno),
                    'candidato': id_candidato,
                    'cidade': cidade_id,
                    'estado': estado_id,
                    'numero_candidato': parsers.parse_numero_candidato(candidatura.numero_candidato),
                    'nome_urna': parsers.parse_nome_urna(candidatura.nome_urna),
                    'partido': partido_id,
                    'legenda_nome': parsers.parse_legenda(candidatura.legenda_nome),
                    'legenda_composicao': parsers.parse_legenda(candidatura.legenda_composicao),
                    'cargo': parsers.parse_cargo(candidatura.cargo),
                    'despesa_maxima': parsers.parse_despesa_maxima(candidatura.despesa_maxima),
                    'situacao_candidatura': parsers.parse_situacao_candidatura(candidatura.situacao_candidatura),
                    'resultado_candidatura': parsers.parse_resultado_candidatura(candidatura.resultado_candidatura)
                }

                try:
                    cursor.execute(query, atributos)
                    self.cnx.commit()
                except:
                    cursor.close()
                    self._handle_exception(exception=e, row=candidatura)

                sys.stdout.write('\r{} de {}'.format((candidatura[0] + 1), total_candidaturas))
                sys.stdout.flush()

                cursor.close()

            sys.stdout.write('\n')
            tempo_final_estado = time.time()
            print('Tempo total desse estado: {:.2f}s \n'.format((tempo_final_estado - tempo_inicio_estado)))

        tempo_final_ano = time.time()
        print('Tempo total em {}: {:.2f}s \n'.format(ano, (tempo_final_ano - tempo_inicio_ano)))

        self.cnx.close()

    def _load(self, tabela):
        query = ("SELECT * FROM " + tabela)
        return pd.read_sql(query, self.cnx)

    def _handle_exception(self, exception, row):
        print(row)
        self.cnx.close()
        raise exception

    def _get_cidade_id(self, cidade, estado_id):
        cidade = self.cidades[
            self.cidades['nome'] == cidade and
            self.cidades['estado_id'] == estado_id]
        if (len(cidade) == 1):
            return cidade['id'].iloc[0]
        else:
            return None
    def _get_eleicao_id(self, ano, descricao):
        eleicao = self.eleicoes[
            self.eleicoes['ano'] == ano and
            self.eleicoes['descricao'] == descricao]
        if (len(eleicao) == 1):
            return eleicao['id'].iloc[0]
        else:
            return None
    def _get_estado_id(self, sigla_estado):
        estado = self.estados[self.estados['sigla'] == sigla_estado]
        if (len(estado) == 1):
            return estado['id'].iloc[0]
        else:
            return None
    def _get_partido_id(self, numero):
        partido = self.partidos[self.partidos['numero'] == numero]
        if (len(partido) == 1):
            return partido['id'].iloc[0]
        else:
            return None

    def _get_colunas_candidatura(self, ano):
        padrao = [
            'NOME_CANDIDATO',
            'DATA_NASCIMENTO',
            'CPF_CANDIDATO',
            'NUM_TITULO_ELEITORAL_CANDIDATO',
            'NOME_MUNICIPIO_NASCIMENTO',
            'SIGLA_UF_NASCIMENTO',
            'CODIGO_NACIONALIDADE',
            'COD_GRAU_INSTRUCAO',
            'CODIGO_OCUPACAO',

            'ANO_ELEICAO',
            'NUM_TURNO',
            'DESCRICAO_ELEICAO',
            'SIGLA_UE',
            'DESCRICAO_UE',
            'SIGLA_UF',
            'CODIGO_CARGO',
            'NUMERO_CANDIDATO',
            'NOME_URNA_CANDIDATO',
            'COD_SITUACAO_CANDIDATURA',
            'NUMERO_PARTIDO',
            'COMPOSICAO_LEGENDA',
            'NOME_LEGENDA',
            'DESPESA_MAX_CAMPANHA',
            'COD_SIT_TOT_TURNO'
        ]

        return padrao if (ano < 2012) else (padrao + ['NM_EMAIL'])
    def _get_map_colunas_candidatura(self, ano):
        sem_email = {
            'NOME_CANDIDATO': 'nome',
            'DATA_NASCIMENTO': 'data_nascimento',
            'CPF_CANDIDATO': 'cpf',
            'NUM_TITULO_ELEITORAL_CANDIDATO': 'titulo_eleitoral',
            'NOME_MUNICIPIO_NASCIMENTO': 'cidade_nascimento',
            'SIGLA_UF_NASCIMENTO': 'sigla_estado_nascimento',
            'CODIGO_NACIONALIDADE': 'nacionalidade',
            'COD_GRAU_INSTRUCAO': 'grau_instrucao',
            'CODIGO_OCUPACAO': 'ocupacao',

            'ANO_ELEICAO': 'ano',
            'NUM_TURNO': 'turno',
            'DESCRICAO_ELEICAO': 'descricao_eleicao',
            'SIGLA_UE': 'sigla_cidade_eleicao',
            'DESCRICAO_UE': 'cidade_eleicao',
            'SIGLA_UF': 'sigla_estado_eleicao',
            'CODIGO_CARGO': 'cargo',
            'NUMERO_CANDIDATO': 'numero_candidato',
            'NOME_URNA_CANDIDATO': 'nome_urna',
            'COD_SITUACAO_CANDIDATURA': 'situacao_candidatura',
            'NUMERO_PARTIDO': 'partido',
            'COMPOSICAO_LEGENDA': 'legenda_composicao',
            'NOME_LEGENDA': 'legenda_nome',
            'DESPESA_MAX_CAMPANHA': 'despesa_maxima',
            'COD_SIT_TOT_TURNO': 'resultado_candidatura'
        }
        com_email = {
            'NOME_CANDIDATO': 'nome',
            'DATA_NASCIMENTO': 'data_nascimento',
            'CPF_CANDIDATO': 'cpf',
            'NUM_TITULO_ELEITORAL_CANDIDATO': 'titulo_eleitoral',
            'NM_EMAIL': 'email',
            'NOME_MUNICIPIO_NASCIMENTO': 'cidade_nascimento',
            'SIGLA_UF_NASCIMENTO': 'sigla_estado_nascimento',
            'CODIGO_NACIONALIDADE': 'nacionalidade',
            'COD_GRAU_INSTRUCAO': 'grau_instrucao',
            'CODIGO_OCUPACAO': 'ocupacao',

            'ANO_ELEICAO': 'ano',
            'NUM_TURNO': 'turno',
            'DESCRICAO_ELEICAO': 'descricao_eleicao',
            'SIGLA_UE': 'sigla_cidade_eleicao',
            'DESCRICAO_UE': 'cidade_eleicao',
            'SIGLA_UF': 'sigla_estado_eleicao',
            'CODIGO_CARGO': 'cargo',
            'NUMERO_CANDIDATO': 'numero_candidato',
            'NOME_URNA_CANDIDATO': 'nome_urna',
            'COD_SITUACAO_CANDIDATURA': 'situacao_candidatura',
            'NUMERO_PARTIDO': 'partido',
            'COMPOSICAO_LEGENDA': 'legenda_composicao',
            'NOME_LEGENDA': 'legenda_nome',
            'DESPESA_MAX_CAMPANHA': 'despesa_maxima',
            'COD_SIT_TOT_TURNO': 'resultado_candidatura'
        }

        return sem_email if (ano < 2012) else com_email


In [108]:
hello = MigracaoDadosTSE()

In [109]:
hello.migrar(2004)

Iniciando ../temp\consulta_cand_2004_AC.txt
Total de candidaturas em 2004: 1511


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().